# ***Bayes Theory applied on medical use case***

In [1]:
import pandas as pd

from BayesNet import BayesNet
from BNReasoner import BNReasoner

# Instantiation of the Medical use case
net = BayesNet()
bn = BNReasoner(net)
net.load_from_bifxml("task3/medical_use_case.BIFXML")

# All nodes in the use case
net.get_all_variables()

['Junk-Food',
 'Stress',
 'Job-loss',
 'Weight-gain',
 'Insulin-resistance',
 'Anxiety',
 'High-blood-pressure',
 'Alcohol-consumption',
 'Trauma',
 'Diabetes',
 'Depression',
 'Stroke']

# **Prior marginal**

---

In the examples below we demonstrate the calculation of prior marginals. Without any evidence, the following prior marginals can be calculated:

Probability of Job-loss

In [2]:
bn.marginal_distribution(["Job-loss"], {})

,Job-loss,p
0,True,0.8
1,False,0.2


Probability of Alcohol-consumption

In [3]:
bn.marginal_distribution(["Alcohol-consumption"], {})

,Alcohol-consumption,p
0,True,0.84
1,False,0.16



Probability of Diabetes-consumption and Anxiety


In [4]:
bn.marginal_distribution(["Diabetes", "Anxiety"], {})

,Diabetes,Anxiety,p
0,True,True,0.605208
1,True,False,0.237363
2,False,True,0.113792
3,False,False,0.043637


# **Posterior marginal**

---

In these examples below we demonstrate the calculation of posterior marginals. Without any evidence, the following prior marginals can be calculated:

The probability of Alcohol consumption if it is known that Job-loss is true. It is clear that the probability of someone consuming alcohol is much higher (0.93) if we know that this person lost his job, than the probability if we don't know whether this person lost his job (0.84), as seen in the prior marginal.

In [5]:
bn.marginal_distribution(["Alcohol-consumption"], {"Job-loss": True})

,Alcohol-consumption,p
0,True,0.93
1,False,0.07


The probability of trauma if we know that Alcohol-consumption is true, is 0.86. 

In [6]:
bn.marginal_distribution(["Trauma"], {"Alcohol-consumption": True})

,Trauma,p
0,True,0.864286
1,False,0.135714


This probability is slightly decreasing if we know that another direct cause of Trauma, namely Stress, is not the case. Then the probability of having a Trauma is 0.7.

In [7]:
bn.marginal_distribution(["Trauma"], {"Alcohol-consumption": True, "Stress": False})

,Trauma,p
0,True,0.7
1,False,0.3


# **MAP**

---

In these examples we show what the most probable instantiation is of a variable in the BN given some evidence.

Looking at Weight-gain, we could take into account the evidence of Junk-food. If we know someone eats junk food, then the instantiantiation of weight-gain is most probably False (0.43 is the maximum value of all probable instantions of Weight-gain)

In [8]:
bn.map_mpe_estimation(pd.Series({"Junk-Food": False}), ["Weight-gain"])

,Weight-gain,p
0,False,0.43


If this person eats junk food, the most probable instantiation of Weight-gain is most probably True (again, considering the instantiation with the highest probability given the evidence).

In [9]:
bn.map_mpe_estimation(pd.Series({"Junk-Food": True}), ["Weight-gain"])

,Weight-gain,p
0,True,0.32


# **MPE**

In the following example, the functionality of the MPE function is showed. MPE is a special case of MAP where no query is given. So, the most probable instantiations of Weight-gain = False and Stress = True are as follows.

In [10]:
bn.map_mpe_estimation(pd.Series({"Weight-gain": False, "Stress": True}))

,Stress,Job-loss,Weight-gain,Depression,High-blood-pressure,Insulin-resistance,Junk-Food,Alcohol-consumption,Trauma,Anxiety,Diabetes,Stroke,p
0,True,True,False,True,True,False,False,True,True,True,True,True,0.059793


This is indeed the most probable instantiation of both Weight-gain and Stress, since this combination has the highest probability out of a joint distribution:

In [11]:
bn.joint_probability(E={"Weight-gain": False, "Stress": True})

,Junk-Food,Job-loss,Insulin-resistance,Anxiety,High-blood-pressure,Alcohol-consumption,Trauma,Diabetes,Depression,Stroke,p
0,False,True,True,True,True,True,True,True,True,True,5.144954e-03
1,False,True,True,True,True,True,True,True,True,False,1.714985e-03
2,False,True,True,True,True,True,True,True,False,True,5.716616e-04
3,False,True,True,True,True,True,True,True,False,False,1.905539e-04
4,False,True,True,True,True,True,True,False,True,True,9.079331e-04
...,...,...,...,...,...,...,...,...,...,...,...
507,False,False,False,False,False,False,False,True,False,False,2.793477e-05
508,False,False,False,False,False,False,False,False,True,True,1.531152e-08
509,False,False,False,False,False,False,False,False,True,False,4.593456e-08
510,False,False,False,False,False,False,False,False,False,True,1.515840e-06


# **D-separation**

In the examples below, the functionality of d-separation is shown. Job loss is not independent of trauma if alcohol-consumption is known. That is, because there is an active path from Job-loss to Trauma through Stress.

In [12]:
bn.d_separation_with_pruning(["Job-loss"], ["Alcohol-consumption"], ["Trauma"])

AttributeError: 'BNReasoner' object has no attribute 'd_separation_with_pruning'

 But, if Stress is added to the evidence, job loss is independent of trauma. 

In [ ]:
bn.d_separation_with_pruning(
    ["Job-loss"], ["Alcohol-consumption", "Stress"], ["Trauma"]
)

A convergent valve ->W<- is only blocked if it is not in the evidence set and neither are its descendents. For example, Job-loss is not d-seperated from Stress by Alcohol-consumption:

In [ ]:
bn.d_separation_with_pruning(["Job-loss"], ["Alcohol-consumption"], ["Stress"])

But if Alcohol consumption or any if its descendents (Trauma) is not given, the path to Stress is blocked:

In [ ]:
bn.d_separation_with_pruning(["Job-loss"], [], ["Stress"])

In [ ]:
bn.d_separation_with_pruning(["Job-loss"], ["Trauma"], ["Stress"])